# Data Management Report
#### **Master Degree in Data Science**

### ***Project 1***: **Querying a database**

16 January 2020    


- **Candidate 1: Christian Riccio P37000002**
- **Candidate 2: Giacomo Matrone P37000011**

# Introduction

The following report comes from an analysis of mock-up database that was downloaded from Oracle.com. 
The data regards the operativity of a multinational firm that sells world wide computer parts and has different warehouses all over the world.
Our analysis has been focused in discovering and exploiting actionable informations about the activity of the firm.

In particular, we discovered the following results:

- The average profit per unit by item category and warehouse's cities;
- Gross profit mean and variance.


In [85]:
import cx_Oracle 
import pandas as pd
import matplotlib
import os 


# Connection to the Database

In this section we define the variables used in order to set the connection to our SQL database, via cx_Oracle library.
We also define a function in order to automate queries execution and we present the results in pandas dataframe format.

In [30]:
user = "SYS as SYSDBA" 
password = "Christian21"
host = '127.0.0.1'
port = '1521'
sid = 'orcl'
conn_str = 'jdbc:oracle:thin:@{0}:{1}:{2}'.format(host,port,sid)


In [31]:
conn_str 

'jdbc:oracle:thin:@127.0.0.1:1521:orcl'

In [32]:
dsn = cx_Oracle.makedsn(host,port,sid)

In [33]:
dsn

'(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=127.0.0.1)(PORT=1521))(CONNECT_DATA=(SID=orcl)))'

In [34]:
connection = cx_Oracle.connect(user,password,dsn,cx_Oracle.SYSDBA)

In [35]:
cursor = connection.cursor()

## cx_Oracle trial

Here is performed an example query with cx_Oracle library.

In [36]:
query='SELECT * FROM REGIONS'

In [37]:
for row in cursor.execute(query):
    print(row)

(1, 'Europe')
(2, 'Americas')
(3, 'Asia')
(4, 'Middle East and Africa')


In [38]:
cursor.execute(query)
data = cursor.fetchall()

In [39]:
data

[(1, 'Europe'), (2, 'Americas'), (3, 'Asia'), (4, 'Middle East and Africa')]

## Metadata exploration 

In order to understand metadata, we proceed in investigating `cursor.description`, which tells us different informations, like: 

- Column name,
- Variable type,
- Eventual constraints.


In [40]:
cursor.description

[('REGION_ID', cx_Oracle.NUMBER, 127, None, 0, -127, 0),
 ('REGION_NAME', cx_Oracle.STRING, 50, 50, None, None, 0)]

In [41]:
regions = pd.DataFrame(cursor.execute(query)) 

In [42]:
regions

,0,1
0,1,Europe
1,2,Americas
2,3,Asia
3,4,Middle East and Africa


## sql_data function 

Our function allows us to perform fast queries whose results are presented in pandas dataframe.
Column names are retrived from cursor metadata and are then inserted in the pandas dataframe.
This function estabilishes a connection for each call to then close the same. This avoids service hanging.

*Why pandas?*

Pandas is a python library written in C that allows fast data munging and wrangling.

In [43]:
def sql_data(sql_statement,user,password,host,port,sid,col_names=True):
    dsn = cx_Oracle.makedsn(host,port,sid)
    connection = cx_Oracle.connect(user,password,dsn,cx_Oracle.SYSDBA)
    cursor = connection.cursor()
    
    if col_names: 
        cursor.execute(sql_statement)
        col_names = [el[0] for el in cursor.description]
        df = pd.DataFrame(cursor.fetchall(), columns = col_names)
    else: 
        df = pd.DataFrame(cursor.execute(sql_statement)) 
    connection.close()
    return df


### A first example

Here we run a simple trial of our function and, as you can see, it produces a pandas dataframe with all the data in it.
Data comes from our SQL DB and is presented in pandas format.

In [44]:
sql_data('SELECT * FROM REGIONS',user,password,host,port,sid)

,REGION_ID,REGION_NAME
0,1,Europe
1,2,Americas
2,3,Asia
3,4,Middle East and Africa


### Initializing variables 

Here we define a list of all the tables' names and then we call SQL data first in a list comprehension and then in a dictionary comprehension. Those allow us to explore faster the database and has been chosen since the dimension of the DB is really small. 
The dictionary will have as keys the name of single tables of the DB and as values the pandas object obtained via `sql_function()`.

In [45]:
table_list = ['contacts', 'countries', 'employees', 'inventories', 'locations',
      'order_items', 'orders', 'products', 'warehouses']


In [46]:
all_data = [sql_data("SELECT * FROM %s"%(el),user,password,host,port,sid) for el in table_list]


In [71]:
all_data[6]

,ORDER_ID,CUSTOMER_ID,STATUS,SALESMAN_ID,ORDER_DATE
0,105,1,Pending,54.0,2016-11-17
1,44,2,Pending,55.0,2017-02-20
2,5,5,Canceled,56.0,2017-04-09
3,4,8,Shipped,59.0,2015-04-09
4,2,4,Shipped,NaN,2015-04-26
5,3,5,Shipped,NaN,2017-04-26
6,6,6,Shipped,NaN,2015-04-09
7,7,7,Shipped,NaN,2017-02-15
8,8,8,Shipped,NaN,2017-02-14
9,9,9,Shipped,NaN,2017-02-14


In [48]:
all_data_dict = {el:sql_data("SELECT * FROM %s"%(el),user,password,host,port,sid) for el in table_list}

In [49]:
all_data_dict.keys()

dict_keys(['contacts', 'countries', 'employees', 'inventories', 'locations', 'order_items', 'orders', 'products', 'warehouses'])

In [81]:
all_data_dict[list(all_data_dict.keys())[6]]

,ORDER_ID,CUSTOMER_ID,STATUS,SALESMAN_ID,ORDER_DATE
0,105,1,Pending,54.0,2016-11-17
1,44,2,Pending,55.0,2017-02-20
2,5,5,Canceled,56.0,2017-04-09
3,4,8,Shipped,59.0,2015-04-09
4,2,4,Shipped,NaN,2015-04-26
5,3,5,Shipped,NaN,2017-04-26
6,6,6,Shipped,NaN,2015-04-09
7,7,7,Shipped,NaN,2017-02-15
8,8,8,Shipped,NaN,2017-02-14
9,9,9,Shipped,NaN,2017-02-14


###  PRODUCT_NAME summary

We used `pandas.Series().value_counts()` method to understand how many times the single products repeat in the data frame resulting from the following query:

```SQL
SELECT *
   FROM products 
       INNER JOIN inventories 
           ON products.PRODUCT_ID = inventories.PRODUCT_ID 
```

As we can see the result gives us the absolute frequency of the single product name repetition.

In [51]:
sql_data("SELECT * FROM products INNER JOIN inventories ON products.PRODUCT_ID = inventories.PRODUCT_ID ",user,password,host,port,sid).loc[:,"PRODUCT_NAME"].value_counts()

G.Skill Ripjaws V Series       58
Corsair Vengeance LPX          40
G.Skill Trident Z              38
Corsair Dominator Platinum     28
Kingston                       20
                               ..
Asus TUF X299 MARK 1            1
ASRock Z270 SuperCarrier        1
ASRock EP2C612 WS               1
Western Digital WDS256G1X0C     1
Western Digital WD20EZRZ        1
Name: PRODUCT_NAME, Length: 173, dtype: int64

### Joining data 

We then joined data in order to obtained a bigger picture of the avaiable DB, by executing the query:

```SQL
SELECT * 
    FROM products 
        INNER JOIN inventories 
            ON products.PRODUCT_ID = inventories.PRODUCT_ID 
                WHERE PRODUCT_NAME 
                    LIKE '%Xeon%'
```

With this query we combined informations from products with the inventories. This allows us to link further up to location and countries, as it will be seen in the next steps. 
Please notice that we restricted, for this example purpose, the result only to Xenon processors.

In [52]:
sql_data("SELECT * FROM products INNER JOIN inventories ON products.PRODUCT_ID = inventories.PRODUCT_ID WHERE PRODUCT_NAME LIKE '%Xeon%'",user,password,host,port,sid).head()

,PRODUCT_ID,PRODUCT_NAME,DESCRIPTION,STANDARD_COST,LIST_PRICE,CATEGORY_ID,PRODUCT_ID,WAREHOUSE_ID,QUANTITY
0,211,Intel Xeon E5-2650,"Speed:2.0GHz,Cores:8,TDP:95W",869.03,1064.99,1,211,8,123
1,212,Intel Xeon E5-2680 V4,"Speed:2.4GHz,Cores:14,TDP:120W",1365.13,1639.99,1,212,8,123
2,218,Intel Xeon E5-2660 V4,"Speed:2.0GHz,Cores:14,TDP:105W",1194.03,1388.89,1,218,8,126
3,241,Intel Xeon E5-2699 V4 (OEM/Tray),"Speed:2.2GHz,Cores:22,TDP:145W",1535.62,1756.00,1,241,8,121
4,242,Intel Xeon E5-1680 V3 (OEM/Tray),"Speed:3.2GHz,Cores:8,TDP:140W",1519.85,1751.99,1,242,8,121


### Null value search

We have also performed a brief search about `NULL` values in employees table, in phone column. Our discovery was pretty obvious.
As you might have guessed, there are no missing values.

In [53]:
sql_data(" SELECT * FROM employees WHERE PHONE IS NULL",user,password,host,port,sid)

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE,HIRE_DATE,MANAGER_ID,JOB_TITLE


### Job title mapping

Here is presented the list of unique job titles avaiable in employees table.


In [55]:
sql_data("SELECT DISTINCT JOB_TITLE FROM employeEs ORDER BY JOB_TITLE ASC",user,password,host,port,sid,col_names=True)

,JOB_TITLE
0,Accountant
1,Marketing Manager
2,Programmer
3,Purchasing Clerk
4,Sales Manager
5,Sales Representative
6,Shipping Clerk
7,Stock Clerk
8,Stock Manager


# General analysis 

From this point onward, we present our analysis on the database. We, as illustrated below, create first a unique dataframe whose columns give us informations about:

- The product (PRODUCT_ID, PRODUCT_NAME, DESCRIPTION, etc.),
- The inventory (units in storage or price etc.),
- The warehouse,
- The geographical dislocation.

The query launched is the following:
```SQL
SELECT * 
    FROM products 
        INNER JOIN inventories 
            ON products.PRODUCT_ID = inventories.PRODUCT_ID 
        INNER JOIN warehouses 
            ON inventories.WAREHOUSE_ID=warehouses.WAREHOUSE_ID 
        INNER JOIN locations 
            ON warehouses.LOCATION_ID=locations.LOCATION_ID 
        INNER JOIN countries 
            ON locations.COUNTRY_ID = countries.COUNTRY_ID
```


In [56]:
sql_data("SELECT * FROM products INNER JOIN inventories ON products.PRODUCT_ID = inventories.PRODUCT_ID INNER JOIN warehouses ON inventories.WAREHOUSE_ID=warehouses.WAREHOUSE_ID INNER JOIN locations ON warehouses.LOCATION_ID=locations.LOCATION_ID INNER JOIN countries ON locations.COUNTRY_ID = countries.COUNTRY_ID",user,password,host,port,sid)

,PRODUCT_ID,PRODUCT_NAME,DESCRIPTION,STANDARD_COST,LIST_PRICE,CATEGORY_ID,PRODUCT_ID,WAREHOUSE_ID,QUANTITY,WAREHOUSE_ID,...,LOCATION_ID,LOCATION_ID,ADDRESS,POSTAL_CODE,CITY,STATE,COUNTRY_ID,COUNTRY_ID,COUNTRY_NAME,REGION_ID
0,210,Intel Core i9-7900X,"Speed:3.3GHz,Cores:10,TDP:140W",855.82,1029.99,1,210,8,122,8,...,11,11,40-5-12 Laogianggen,190518,Beijing,None,CN,CN,China,3
1,211,Intel Xeon E5-2650,"Speed:2.0GHz,Cores:8,TDP:95W",869.03,1064.99,1,211,8,123,8,...,11,11,40-5-12 Laogianggen,190518,Beijing,None,CN,CN,China,3
2,212,Intel Xeon E5-2680 V4,"Speed:2.4GHz,Cores:14,TDP:120W",1365.13,1639.99,1,212,8,123,8,...,11,11,40-5-12 Laogianggen,190518,Beijing,None,CN,CN,China,3
3,214,Intel Core i7-5960X,"Speed:3.0GHz,Cores:8,TDP:140W",865.59,1009.79,1,214,8,124,8,...,11,11,40-5-12 Laogianggen,190518,Beijing,None,CN,CN,China,3
4,216,MSI GTX 1080 TI LIGHTNING X,"Chipset:GeForce GTX 1080 Ti,Memory:11GBCore Cl...",742.94,863.98,2,216,8,125,8,...,11,11,40-5-12 Laogianggen,190518,Beijing,None,CN,CN,China,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107,201,Kingston,"Speed:DDR3-1600,Type:240-pin DIMM,CAS:11Module...",566.98,653.50,5,201,8,142,8,...,11,11,40-5-12 Laogianggen,190518,Beijing,None,CN,CN,China,3
1108,203,Kingston,"Speed:DDR3-1333,Type:240-pin DIMM,CAS:9Module:...",556.84,671.38,5,203,8,142,8,...,11,11,40-5-12 Laogianggen,190518,Beijing,None,CN,CN,China,3
1109,204,G.Skill Ripjaws V Series,"Speed:DDR4-3200,Type:288-pin DIMM,CAS:15Module...",546.64,686.99,5,204,8,142,8,...,11,11,40-5-12 Laogianggen,190518,Beijing,None,CN,CN,China,3
1110,205,G.Skill Trident X,"Speed:DDR3-3100,Type:240-pin DIMM,CAS:12Module...",507.32,649.99,5,205,8,142,8,...,11,11,40-5-12 Laogianggen,190518,Beijing,None,CN,CN,China,3


In the following example, we explore the managed orders from warehouses, in order to obtain details about their status. 

```SQL
SELECT WAREHOUSE_NAME, 
    COUNT(STATUS) 
        AS number_orders, 
       STATUS 
      FROM
 (SELECT * 
     FROM (
         orders 
             INNER JOIN order_items 
                 ON orders.ORDER_ID = order_items.ORDER_ID 
             INNER JOIN inventories 
                 ON order_items.PRODUCT_ID=inventories.PRODUCT_ID 
             INNER JOIN warehouses 
                 ON inventories.WAREHOUSE_ID=warehouses.WAREHOUSE_ID 
             INNER JOIN locations 
                 ON warehouses.LOCATION_ID=locations.LOCATION_ID
           )) 
           WHERE 
               STATUS='Shipped'  
           GROUP BY
               (WAREHOUSE_NAME,STATUS)
```

In [82]:
sql_data(" SELECT WAREHOUSE_NAME, COUNT(STATUS) AS number_orders, STATUS FROM( SELECT * FROM (orders INNER JOIN order_items ON orders.ORDER_ID = order_items.ORDER_ID INNER JOIN inventories ON order_items.PRODUCT_ID=inventories.PRODUCT_ID INNER JOIN warehouses ON inventories.WAREHOUSE_ID=warehouses.WAREHOUSE_ID INNER JOIN locations ON warehouses.LOCATION_ID=locations.LOCATION_ID))  WHERE STATUS='Shipped'  GROUP BY(WAREHOUSE_NAME,STATUS)" ,user,password,host,port,sid)

,WAREHOUSE_NAME,NUMBER_ORDERS,STATUS
0,"Southlake, Texas",15,Shipped
1,Sydney,108,Shipped
2,Toronto,49,Shipped
3,Bombay,71,Shipped
4,Mexico City,45,Shipped
5,Beijing,95,Shipped
6,San Francisco,88,Shipped
7,New Jersey,21,Shipped
8,"Seattle, Washington",59,Shipped


## Average profit calculation

We grouped all the data retrived from the former query, in order to calculate the average profit by `CITY` and `CATEGORY_ID`. The query we launched is the following:
```SQL
SELECT CITY,CATEGORY_ID, AVG(LIST_PRICE-STANDARD_COST) AS avg_profit 
    FROM 
    (SELECT * 
         FROM products 
             INNER JOIN inventories 
                 ON products.PRODUCT_ID = inventories.PRODUCT_ID 
             INNER JOIN warehouses 
                 ON inventories.WAREHOUSE_ID=warehouses.WAREHOUSE_ID 
             INNER JOIN locations 
                 ON warehouses.LOCATION_ID=locations.LOCATION_ID 
             INNER JOIN countries 
                 ON locations.COUNTRY_ID = countries.COUNTRY_ID) 
    GROUP BY (CITY, CATEGORY_ID)
```

We finally use the `pandas.Series().replace()` function in order to map the CATEGORY_ID column from integer to a string containing an acronymic description.

In [ ]:
sql_data("SELECT CITY,CATEGORY_ID, AVG(LIST_PRICE-STANDARD_COST) AS avg_profit FROM (SELECT * FROM products INNER JOIN inventories ON products.PRODUCT_ID = inventories.PRODUCT_ID INNER JOIN warehouses ON inventories.WAREHOUSE_ID=warehouses.WAREHOUSE_ID INNER JOIN locations ON warehouses.LOCATION_ID=locations.LOCATION_ID INNER JOIN countries ON locations.COUNTRY_ID = countries.COUNTRY_ID) GROUP BY (CITY, CATEGORY_ID) ",user,password,host,port,sid).replace({1:"CPU",2:"GPU",4:"MoBo",5:"HDD/RAM"})

### Bonus analysis 

In the end, we give an insight about gross income distribution by category, by executing the following query:
```SQL
SELECT CATEGORY_ID, avg(QUANTITY*UNIT_PRICE) AS avg_gross_income, variance(QUANTITY*UNIT_PRICE) AS var_gross_income 
    FROM 
    (SELECT * 
         FROM ORDER_ITEMS 
             LEFT JOIN products 
                 ON order_items.PRODUCT_ID=products.PRODUCT_ID ) 
    GROUP BY CATEGORY_ID
```
As it is possible to notice, the CPU and GPU show the greatest mean gross income and has the greatest variability.
The query relates `order_items` with `products` tables. 

In [84]:
sql_data("SELECT CATEGORY_ID, avg(QUANTITY*UNIT_PRICE) AS avg_gross_income, variance(QUANTITY*UNIT_PRICE) AS var_gross_income FROM (SELECT * FROM ORDER_ITEMS LEFT JOIN products ON order_items.PRODUCT_ID=products.PRODUCT_ID ) GROUP BY CATEGORY_ID",user,password,host,port,sid).replace({1:"CPU",2:"GPU",4:"MoBo",5:"HDD/RAM"})

,CATEGORY_ID,AVG_GROSS_INCOME,VAR_GROSS_INCOME
0,CPU,126050.270629,6.126451e+09
1,GPU,131309.720190,1.106244e+10
2,MoBo,35640.711232,4.530642e+08
3,HDD/RAM,53209.737034,3.150839e+09


Here, the above query has been modified in order to obtain informations about the maximum income due by each product. The request pass trough the following query:
```SQL
SELECT CATEGORY_ID, max(QUANTITY*UNIT_PRICE) AS max_gross_income 
    FROM 
    (SELECT * 
         FROM ORDER_ITEMS 
             LEFT JOIN products 
                 ON order_items.PRODUCT_ID=products.PRODUCT_ID ) 
    GROUP BY CATEGORY_ID
```


In [83]:
sql_data("SELECT CATEGORY_ID, max(QUANTITY*UNIT_PRICE) AS max_gross_income FROM (SELECT * FROM ORDER_ITEMS LEFT JOIN products ON order_items.PRODUCT_ID=products.PRODUCT_ID ) GROUP BY CATEGORY_ID",user,password,host,port,sid).replace({1:"CPU",2:"GPU",4:"MoBo",5:"HDD/RAM"})

,CATEGORY_ID,MAX_GROSS_INCOME
0,CPU,328038.42
1,GPU,600155.00
2,MoBo,120521.73
3,HDD/RAM,603023.32
